### 1. Sales Rollup by Territory Hierarchy

**Functional Specification**
- Join Sales and Territory tables.
- Select Group, CountryRegionCode, and Name (Region).
- Use ROLLUP on (Group, Country, Region).
- Sum SalesYTD.
- This provides totals at the Region level, Country level, Group level, and Grand Total.

In [ ]:
SELECT 
    [Group],
    CountryRegionCode,
    Name AS Region,
    SUM(SalesYTD) AS TotalSalesYTD
FROM Sales.SalesTerritory
GROUP BY ROLLUP ([Group], CountryRegionCode, Name)
ORDER BY [Group], CountryRegionCode, Name;

### 2. Inventory CUBE by Shelf and Location

**Functional Specification**
- Query Production.ProductInventory.
- Use CUBE on Shelf and LocationID.
- Sum Quantity.
- This generates totals for every combination: per Shelf, per Location, per Shelf-Location pair, and Grand Total.

In [ ]:
SELECT 
    Shelf,
    LocationID,
    SUM(Quantity) AS TotalQty
FROM Production.ProductInventory
GROUP BY CUBE (Shelf, LocationID)
ORDER BY Shelf, LocationID;

### 3. Specific Groupings: Product Counts (Replacement for Employee)

**Functional Specification**
- Query Production.Product (Substituting HumanResources.Employee to avoid CLR error).
- Use GROUPING SETS.
- Define sets: (ProductLine), (Class), and () (Grand Total).
- Count ProductID.
- This creates three distinct reports in one query result.

In [ ]:
SELECT 
    ProductLine,
    Class,
    COUNT(ProductID) AS ProductCount
FROM Production.Product
WHERE ProductLine IS NOT NULL
GROUP BY GROUPING SETS (
    (ProductLine),
    (Class),
    ()
)
ORDER BY ProductLine, Class;

### 4. Time Hierarchies: Sales by Year, Month, Day

**Functional Specification**
- Extract Year, Month, and Day from OrderDate.
- Use ROLLUP(Year, Month, Day).
- Sum TotalDue.
- Allows drilling down from annual sales to daily sales.

In [ ]:
WITH SalesByDate AS (
    SELECT 
        YEAR(OrderDate)  AS [Year],
        MONTH(OrderDate) AS [Month],
        DAY(OrderDate)   AS [Day],
        TotalDue
    FROM Sales.SalesOrderHeader
)
SELECT 
    [Year], 
    [Month], 
    [Day],
    SUM(TotalDue) AS TotalSales
FROM SalesByDate
GROUP BY ROLLUP ([Year], [Month], [Day])
ORDER BY [Year], [Month], [Day];

### 5. Product Categorization Rollup

**Functional Specification**
- Join Product, Subcategory, and Category tables.
- Use ROLLUP(CategoryName, SubcategoryName).
- Count Products.
- Show how many products exist in each category and subcategory.

In [ ]:
WITH ProductRollup AS (
    SELECT 
        c.Name  AS Category,
        s.Name  AS Subcategory
    FROM Production.Product AS p
    JOIN Production.ProductSubcategory AS s
        ON p.ProductSubcategoryID = s.ProductSubcategoryID
    JOIN Production.ProductCategory AS c
        ON s.ProductCategoryID = c.ProductCategoryID
)
SELECT 
    Category,
    Subcategory,
    COUNT(*) AS ProductCount
FROM ProductRollup 
GROUP BY ROLLUP (Category, Subcategory)
ORDER BY Category, Subcategory;

### 6. Identifying Grouping Levels (GROUPING_ID)

**Functional Specification**
- Use the query from Proposition 1 (Sales Rollup).
- Add GROUPING_ID(st.[Group], st.CountryRegionCode, st.Name) to the select list.
- This binary bitmask helps identifying which row represents a subtotal versus a detailed row programmatically.

In [ ]:
WITH RollupData AS (
    SELECT 
        st.[Group],
        st.CountryRegionCode,
        st.Name AS Region,
        SUM(st.SalesYTD) AS TotalSales,
        GROUPING_ID(st.[Group], st.CountryRegionCode, st.Name) AS GroupingLevel
    FROM Sales.SalesTerritory AS st
    GROUP BY ROLLUP(st.[Group], st.CountryRegionCode, st.Name)
)
SELECT *
FROM RollupData
ORDER BY [Group], CountryRegionCode, Region;

### 7. Purchase Orders by Employee and Vendor (CUBE)

**Functional Specification**
- Query Purchasing.PurchaseOrderHeader.
- Use CUBE(EmployeeID, VendorID).
- Sum TotalDue.
- Analyze which employees buy from which vendors, total spend per employee, and total spend per vendor.

In [ ]:
WITH PurchaseOrder AS (
    SELECT 
        EmployeeID,
        VendorID,
        TotalDue
    FROM Purchasing.PurchaseOrderHeader
)
SELECT 
    EmployeeID,
    VendorID,
    SUM(TotalDue) AS TotalSpend  
FROM PurchaseOrder
GROUP BY CUBE (EmployeeID, VendorID)
ORDER BY EmployeeID, VendorID;

### 8. Customer Demographics Grouping

**Functional Specification**
- Query Person.Person (assumes demographics or simple fields available).
- Use GROUPING SETS on (PersonType) and (EmailPromotion).
- Count entities.
- Provides distinct breakdowns of person types (Employee, SC, VC) and email marketing preferences.

In [ ]:
SELECT 
    PersonType,
    EmailPromotion,
    COUNT(*) AS TotalCount
FROM Person.Person
GROUP BY GROUPING SETS (
    (PersonType),
    (EmailPromotion),
    ()
)
ORDER BY PersonType, EmailPromotion;

### 9. Sales Reason and Territory Combination

**Functional Specification**
- Join headers, territories, and sales reasons.
- Use GROUPING SETS.
- Set 1: (TerritoryName).
- Set 2: (SalesReasonName).
- Sum TotalDue.
- Compare revenue driven by location vs revenue driven by marketing reason.

In [ ]:
SELECT 
    st.Name AS TerritoryName,
    sr.Name AS SalesReasonName,
    SUM(soh.TotalDue) AS TotalDue
FROM Sales.SalesOrderHeader AS soh
JOIN Sales.SalesOrderHeaderSalesReason AS srsr
    ON soh.SalesOrderID = srsr.SalesOrderID
JOIN Sales.SalesReason AS sr
    ON srsr.SalesReasonID = sr.SalesReasonID
JOIN Sales.SalesTerritory AS st
    ON soh.TerritoryID = st.TerritoryID
GROUP BY GROUPING SETS (
    (st.Name),
    (sr.Name)
)
ORDER BY TerritoryName, SalesReasonName;

### 10. Yearly Sales by Ship Method Rollup

**Functional Specification**
- Join SalesOrderHeader and ShipMethod.
- Use ROLLUP(Year, ShipMethodName).
- Count Orders.
- Show order volume per year, broken down by how they were shipped.

In [ ]:
WITH ShipSales AS (
    SELECT
        YEAR(soh.OrderDate) AS Year,
        sm.Name AS ShipMethodName,
        soh.SalesOrderID
    FROM Sales.SalesOrderHeader AS soh
    JOIN Purchasing.ShipMethod  AS sm
        ON soh.ShipMethodID = sm.ShipMethodID
)
SELECT
    Year,
    ShipMethodName,
    COUNT(SalesOrderID) AS OrderCount
FROM ShipSales
GROUP BY ROLLUP (Year, ShipMethodName)
ORDER BY Year, ShipMethodName;